In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import PIL
from PIL import Image
from sklearn.preprocessing import StandardScaler

In [2]:
label = pd.read_csv("./Penyisihan Hology Data Mining/train.csv")
path_image = './Penyisihan Hology Data Mining/train'
# function untuk membuka image
def buka_id_image(image_id, path_image):
    try:
        image_path = os.path.join(path_image, f"{image_id}.jpg")
        lek = Image.open(image_path)
        return lek
    #jika nanti file format png tidak ditemukan langsung dialihkan untuk 
    #di convert ke rgb(karena fitur rgb hanya di jpg) lalu di save ke direktori yang sama ke file model berada
    except FileNotFoundError:
        image_path1 = os.path.join(path_image, f"{image_id}.png")
        image_conversion = Image.open(image_path1).convert('RGB')
        image_conversion.save(f"{image_id}.jpg", 'JPEG')
        return image_conversion
label.head()

,id,jenis,warna
0,1,1,1
1,2,0,2
2,3,1,3
3,4,1,1
4,5,0,4


In [3]:
#ngubah setiap pixel dari biner(0,255) yang bernilai putih ke skala(0,1)
#umumnya disebut normalisasi satu skala dibuat dalam pixel
image_train = []
image_label = []
def extract_roi(image, box=(20, 40, 160, 150)):
    return image.crop(box)
img=Image.open("C:/Users/DANIEL/OneDrive/文件/PYTHON/datamining/Penyisihan Hology Data Mining/test/783.jpg").convert("RGB")
img1=extract_roi(img)
img1.show()
def image_to_array(image, image_train, size=(200, 200)):
    image=extract_roi(image)
    image_resized = image.resize(size)  # Resize image
    t = np.array(image_resized).astype(np.float64)
    array = t / 255.0  # Normalize pixel values
    image_train.append(array)

In [4]:
# print(image_train)
for i, row in label.iterrows():
    image_id = row['id']
    picture = buka_id_image(image_id, path_image)
    image_to_array(picture, image_train)
    image_label.append([row['jenis'],row['warna']])

X = np.array(image_train)
y = np.array(image_label,dtype=float)

In [8]:
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.cluster import KMeans
X_flattened = X.reshape(X.shape[0], -1)
model_clustering=KMeans(algorithm='lloyd',n_clusters=2)
Xb,yn=model_clustering.fit(X,y)
# n_estimators=200,criterion='gini',random_state=42
rbm = BernoulliRBM(n_components=100, learning_rate=0.01, n_iter=100, random_state=42)

# Initialize a logistic regression model
logistic = LogisticRegression(max_iter=1000)

# Use a MultiOutputClassifier to handle multiple binary outputs
multi_output_classifier = MultiOutputClassifier(logistic)

# Create a pipeline with RBM and the multi-output logistic regression
rbm_classifier = Pipeline(steps=[('rbm', rbm), ('logistic', multi_output_classifier)])

# Train the model
rbm_classifier.fit(X_flattened, y)

ValueError: Found array with dim 4. KMeans expected <= 2.

In [7]:
image_test=[]
writing=pd.read_csv('./Penyisihan Hology Data Mining/sample_submission.csv')

In [8]:
path_image3='./Penyisihan Hology Data Mining/test'
for i, row in writing.iterrows():
    image_id = row['id']
    picture = buka_id_image(image_id, path_image3)
    image_to_array(picture, image_test)


In [11]:
X_test=np.array(image_test)
XF_test=X_test.reshape(X_test.shape[0], -1)
t=rbm_classifier.predict(XF_test)
print(t[15])

[0. 0.]


In [12]:
cun=t.astype(int)
for i in range(len(writing)):
    writing.loc[i, 'jenis'] = cun[i][0]  # Prediksi jenis
    writing.loc[i, 'warna'] = cun[i][1]  # Prediksi warna
writing.to_csv('./Penyisihan Hology Data Mining/submission12.csv', index=False)

In [1]:
df1=pd.read_csv('C:/Users/DANIEL/OneDrive/文件/PYTHON/datamining/Penyisihan Hology Data Mining/submission2.csv')

NameError: name 'pd' is not defined